In [11]:
import numpy as np 
import sympy.geometry as sp
import cProfile
from area_intersect import polygon_intersect_area
from point_in_polygon import polygon_inside_points
from segment_polygon_intersection import segment_polygon_intersection
from poly_poly_intersect import polygon_area, gift_wrapper

In [2]:
%load_ext line_profiler

In [26]:
def rectangle_intersection(P, Q):
    
    if len(P) != 4 or len(Q) != 4:
        raise ValueError('There are non rectangles in the input!')
    
    # Test for convexity
    P = gift_wrapper(P)
    Q = gift_wrapper(Q)
    assert len(P) == 4, "First polygon is not convex"
    assert len(Q) == 4, "Second polygon is not convex"
    
    I = []
        
    # Search for border intersection
    for i in range(4):
        j = (i+1)%4
        seg = [Q[i], Q[j]]
        point = segment_polygon_intersection(P, seg)
        
        if len(point)>0:
            I.extend(point)
    
    # Points of rectangle vertex inside the other
    s = polygon_inside_points(P, Q)
    t = polygon_inside_points(Q, P)

    # Concatenate points 
    if len(t) > 0:
        I = np.concatenate([I, t], axis=0)
    if len(s) > 0:
        I = np.concatenate([I, s], axis=0)
    
    return polygon_area(I)

In [17]:
rec1 = np.array(([0,0], [5,0], [5,5], [0,5]))
rec2 = np.array(([3, -1], [7, 3], [2, 8], [-2, 4]))

In [18]:
R1 = sp.Polygon(*rec1)
R2 = sp.Polygon(*rec2)

## Sympy

In [19]:
I = sp.intersection(R1, R2)
A = sp.Polygon(*I).area
A

15

In [20]:
smpt = %timeit -o sp.Polygon(*sp.intersection(R1, R2)).area

71.4 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Generic Polygon

In [8]:
parea = polygon_intersect_area(rec1, rec2)
parea

(22.5, array([[ 0.,  2.],
        [ 2.,  0.],
        [ 4.,  0.],
        [ 5.,  1.],
        [ 5.,  5.],
        [ 0.,  5.]]))

In [29]:
gnpt = %timeit -o polygon_intersect_area(rec1, rec2)

2.39 ms ± 205 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
%lprun -f polygon_intersect_area polygon_intersect_area(rec1, rec2)

## Rectangle

In [23]:
recta = rectangle_intersection(rec1, rec2)
recta

[array([0, 5])]


22.5

In [30]:
rt = %timeit -o rectangle_intersection(rec1, rec2)

2.18 ms ± 173 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
